In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
with open('intents.json') as file:
    data = json.load(file)

In [4]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay'],
   'responses': ['Hello', 'Hi', 'Hi there']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later', 'Have a nice day', 'Bye! Come back again']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Thanks for the help'],
   'responses': ['Happy to help!',
    'Any time!',
    'My pleasure',
    "You're most welcome!"]},
  {'tag': 'about',
   'patterns': ['Who are you?', 'What are you?', 'Who you are?'],
   'responses': ["I'm Maxx, your bot assistant",
    "I'm Maxx, an Artificial Intelligent bot"]},
  {'tag': 'name',
   'patterns': ['what is your name',
    'what should I call you',
    'whats your name?'],
   'responses': ['You can call me Maxx.',
    "I'm Maxx!",
    'Just call me as Maxx']},
  {'tag': 'help',
   'patterns': ['Could you help me?',
    'give me a hand please',
    'Can you help?',
 

In [5]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [6]:
num_classes

48

In [7]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [8]:
vocab_size = 3000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [9]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Metal device set to: Apple M1


2023-04-30 19:41:50.298079: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-30 19:41:50.298464: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            48000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 48)                816       
                                                                 
Total params: 49,360
Trainable params: 49,360
Non-trainable params: 0
____________________________________________________

In [11]:
epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/550


2023-04-30 19:42:18.909034: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-04-30 19:42:19.079655: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 2s 73ms/step - loss: 3.8721 - accuracy: 0.0256
Epoch 2/550
5/5 [==============================] - 0s 13ms/step - loss: 3.8705 - accuracy: 0.0256
Epoch 3/550
5/5 [==============================] - 0s 13ms/step - loss: 3.8697 - accuracy: 0.0192
Epoch 4/550
5/5 [==============================] - 0s 16ms/step - loss: 3.8688 - accuracy: 0.0321
Epoch 5/550
5/5 [==============================] - 0s 12ms/step - loss: 3.8679 - accuracy: 0.0321
Epoch 6/550
5/5 [==============================] - 0s 10ms/step - loss: 3.8672 - accuracy: 0.0513
Epoch 7/550
5/5 [==============================] - 0s 11ms/step - loss: 3.8663 - accuracy: 0.0705
Epoch 8/550
5/5 [==============================] - 0s 9ms/step - loss: 3.8654 - accuracy: 0.0577
Epoch 9/550
5/5 [==============================] - 0s 8ms/step - loss: 3.8643 - accuracy: 0.0513
Epoch 10/550
5/5 [==============================] - 0s 8ms/step - loss: 3.8633 - accuracy: 0.0449
Epoch 11/550
5/5 [================

5/5 [==============================] - 0s 8ms/step - loss: 2.5786 - accuracy: 0.2244
Epoch 85/550
5/5 [==============================] - 0s 8ms/step - loss: 2.5614 - accuracy: 0.2308
Epoch 86/550
5/5 [==============================] - 0s 8ms/step - loss: 2.5412 - accuracy: 0.2372
Epoch 87/550
5/5 [==============================] - 0s 8ms/step - loss: 2.5236 - accuracy: 0.2628
Epoch 88/550
5/5 [==============================] - 0s 8ms/step - loss: 2.5056 - accuracy: 0.2692
Epoch 89/550
5/5 [==============================] - 0s 8ms/step - loss: 2.4892 - accuracy: 0.2564
Epoch 90/550
5/5 [==============================] - 0s 8ms/step - loss: 2.4693 - accuracy: 0.2564
Epoch 91/550
5/5 [==============================] - 0s 8ms/step - loss: 2.4527 - accuracy: 0.2885
Epoch 92/550
5/5 [==============================] - 0s 8ms/step - loss: 2.4348 - accuracy: 0.2885
Epoch 93/550
5/5 [==============================] - 0s 8ms/step - loss: 2.4191 - accuracy: 0.3077
Epoch 94/550
5/5 [===============

5/5 [==============================] - 0s 7ms/step - loss: 1.3052 - accuracy: 0.7500
Epoch 168/550
5/5 [==============================] - 0s 7ms/step - loss: 1.2905 - accuracy: 0.7436
Epoch 169/550
5/5 [==============================] - 0s 7ms/step - loss: 1.2812 - accuracy: 0.7564
Epoch 170/550
5/5 [==============================] - 0s 8ms/step - loss: 1.2660 - accuracy: 0.7500
Epoch 171/550
5/5 [==============================] - 0s 8ms/step - loss: 1.2543 - accuracy: 0.7628
Epoch 172/550
5/5 [==============================] - 0s 7ms/step - loss: 1.2406 - accuracy: 0.7821
Epoch 173/550
5/5 [==============================] - 0s 8ms/step - loss: 1.2296 - accuracy: 0.7949
Epoch 174/550
5/5 [==============================] - 0s 8ms/step - loss: 1.2170 - accuracy: 0.7628
Epoch 175/550
5/5 [==============================] - 0s 7ms/step - loss: 1.2048 - accuracy: 0.7885
Epoch 176/550
5/5 [==============================] - 0s 8ms/step - loss: 1.1911 - accuracy: 0.8077
Epoch 177/550
5/5 [=====

5/5 [==============================] - 0s 8ms/step - loss: 0.5417 - accuracy: 0.9808
Epoch 250/550
5/5 [==============================] - 0s 7ms/step - loss: 0.5327 - accuracy: 0.9808
Epoch 251/550
5/5 [==============================] - 0s 8ms/step - loss: 0.5290 - accuracy: 0.9808
Epoch 252/550
5/5 [==============================] - 0s 8ms/step - loss: 0.5217 - accuracy: 0.9872
Epoch 253/550
5/5 [==============================] - 0s 8ms/step - loss: 0.5147 - accuracy: 0.9872
Epoch 254/550
5/5 [==============================] - 0s 8ms/step - loss: 0.5082 - accuracy: 0.9872
Epoch 255/550
5/5 [==============================] - 0s 7ms/step - loss: 0.5006 - accuracy: 0.9872
Epoch 256/550
5/5 [==============================] - 0s 7ms/step - loss: 0.4938 - accuracy: 0.9808
Epoch 257/550
5/5 [==============================] - 0s 9ms/step - loss: 0.4884 - accuracy: 0.9808
Epoch 258/550
5/5 [==============================] - 0s 8ms/step - loss: 0.4840 - accuracy: 0.9808
Epoch 259/550
5/5 [=====

5/5 [==============================] - 0s 8ms/step - loss: 0.1823 - accuracy: 1.0000
Epoch 332/550
5/5 [==============================] - 0s 7ms/step - loss: 0.1802 - accuracy: 1.0000
Epoch 333/550
5/5 [==============================] - 0s 7ms/step - loss: 0.1771 - accuracy: 1.0000
Epoch 334/550
5/5 [==============================] - 0s 8ms/step - loss: 0.1748 - accuracy: 1.0000
Epoch 335/550
5/5 [==============================] - 0s 7ms/step - loss: 0.1735 - accuracy: 1.0000
Epoch 336/550
5/5 [==============================] - 0s 7ms/step - loss: 0.1707 - accuracy: 1.0000
Epoch 337/550
5/5 [==============================] - 0s 7ms/step - loss: 0.1686 - accuracy: 1.0000
Epoch 338/550
5/5 [==============================] - 0s 8ms/step - loss: 0.1662 - accuracy: 1.0000
Epoch 339/550
5/5 [==============================] - 0s 8ms/step - loss: 0.1638 - accuracy: 1.0000
Epoch 340/550
5/5 [==============================] - 0s 8ms/step - loss: 0.1617 - accuracy: 1.0000
Epoch 341/550
5/5 [=====

5/5 [==============================] - 0s 8ms/step - loss: 0.0693 - accuracy: 1.0000
Epoch 414/550
5/5 [==============================] - 0s 8ms/step - loss: 0.0680 - accuracy: 1.0000
Epoch 415/550
5/5 [==============================] - 0s 7ms/step - loss: 0.0672 - accuracy: 1.0000
Epoch 416/550
5/5 [==============================] - 0s 8ms/step - loss: 0.0664 - accuracy: 1.0000
Epoch 417/550
5/5 [==============================] - 0s 8ms/step - loss: 0.0657 - accuracy: 1.0000
Epoch 418/550
5/5 [==============================] - 0s 8ms/step - loss: 0.0649 - accuracy: 1.0000
Epoch 419/550
5/5 [==============================] - 0s 8ms/step - loss: 0.0645 - accuracy: 1.0000
Epoch 420/550
5/5 [==============================] - 0s 9ms/step - loss: 0.0638 - accuracy: 1.0000
Epoch 421/550
5/5 [==============================] - 0s 8ms/step - loss: 0.0634 - accuracy: 1.0000
Epoch 422/550
5/5 [==============================] - 0s 7ms/step - loss: 0.0626 - accuracy: 1.0000
Epoch 423/550
5/5 [=====

5/5 [==============================] - 0s 8ms/step - loss: 0.0319 - accuracy: 1.0000
Epoch 496/550
5/5 [==============================] - 0s 7ms/step - loss: 0.0317 - accuracy: 1.0000
Epoch 497/550
5/5 [==============================] - 0s 7ms/step - loss: 0.0314 - accuracy: 1.0000
Epoch 498/550
5/5 [==============================] - 0s 7ms/step - loss: 0.0311 - accuracy: 1.0000
Epoch 499/550
5/5 [==============================] - 0s 7ms/step - loss: 0.0308 - accuracy: 1.0000
Epoch 500/550
5/5 [==============================] - 0s 7ms/step - loss: 0.0306 - accuracy: 1.0000
Epoch 501/550
5/5 [==============================] - 0s 7ms/step - loss: 0.0303 - accuracy: 1.0000
Epoch 502/550
5/5 [==============================] - 0s 7ms/step - loss: 0.0301 - accuracy: 1.0000
Epoch 503/550
5/5 [==============================] - 0s 7ms/step - loss: 0.0298 - accuracy: 1.0000
Epoch 504/550
5/5 [==============================] - 0s 7ms/step - loss: 0.0296 - accuracy: 1.0000
Epoch 505/550
5/5 [=====

In [12]:
# saving model
model.save("chat_model")

import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model/assets
